##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Entity Extraction

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/json_capabilities/Entity_Extraction_JSON.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

You will use Gemini to extract all fields that fit one of the predefined classes and label them.

In [2]:
%pip install -U -q "google-genai>=1.0.0"

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Example

In [4]:
from enum import Enum
from typing_extensions import TypedDict # in python 3.12 replace typing_extensions with typing
from google.genai import types

entity_recognition_text = "John Johnson, the CEO of the Oil Inc. and Coal Inc. companies, has unveiled plans to build a new factory in Houston, Texas."
prompt = f"""
Generate list of entities in text based on the following Python class structure:

class CategoryEnum(str, Enum):
    Person = 'Person'
    Company = 'Company'
    State = 'State'
    City = 'City'

class Entity(TypedDict):
  name: str
  category: CategoryEnum

class Entities(TypedDict):
  entities: list[Entity]

{entity_recognition_text}"""
MODEL_ID = "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0,
        response_mime_type="application/json"
    )
  )

In [5]:
import json

print(json.dumps(json.loads(response.text), indent=4))

{
    "entities": [
        {
            "name": "John Johnson",
            "category": "Person"
        },
        {
            "name": "Oil Inc.",
            "category": "Company"
        },
        {
            "name": "Coal Inc.",
            "category": "Company"
        },
        {
            "name": "Houston",
            "category": "City"
        },
        {
            "name": "Texas",
            "category": "State"
        }
    ]
}


In [9]:
from enum import Enum
from typing_extensions import TypedDict
from google.genai import types

entity_recognition_text = """
During the 2024 Global Tech Forum in San Francisco, California, Dr. Aisha Rahman, Chief AI Officer at Amazon Web Services (AWS),
announced a strategic partnership between AWS and Stanford University to advance ethical AI research.

Meanwhile, Tim Cook, CEO of Apple, revealed plans to open a new R&D center in Apple Valley, Minnesota.
Apple also finalized its acquisition of NeuroSync, a Boston-based neurotech startup.

Elsewhere, Satya Nadella (Microsoft) and Sundar Pichai (Google) discussed AI governance frameworks with officials from the European Union in Brussels, Belgium.

In a parallel announcement, the OpenAI foundation based in San Francisco entered a collaboration with the United Nations (UN) to support AI initiatives for sustainable development in developing countries.

Notably, the joint task force includes scientists from MIT, Carnegie Mellon, and the Indian Institute of Science (IISc), Bengaluru, India.
"""

prompt = f"""
Parse the following text and return a list of entities with rich nested metadata using this structure:

class CategoryEnum(str, Enum):
    Person = 'Person'
    Company = 'Company'
    Organization = 'Organization'
    State = 'State'
    City = 'City'
    Country = 'Country'

class Location(TypedDict):
    city: str | None
    state: str | None
    country: str | None

class Relationship(TypedDict):
    type: str
    target: str

class Entity(TypedDict):
    name: str
    category: CategoryEnum
    aliases: list[str]
    description: str
    location: Location | None
    relationships: list[Relationship]

class Entities(TypedDict):
    entities: list[Entity]

Text:
{entity_recognition_text}
"""


response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0,
        response_mime_type="application/json"
    )
)

import json
print(json.dumps(json.loads(response.text), indent=4))


{
    "entities": [
        {
            "name": "Dr. Aisha Rahman",
            "category": "Person",
            "aliases": [],
            "description": "Chief AI Officer at Amazon Web Services (AWS)",
            "location": null,
            "relationships": [
                {
                    "type": "works for",
                    "target": "Amazon Web Services"
                },
                {
                    "type": "announced partnership with",
                    "target": "Stanford University"
                }
            ]
        },
        {
            "name": "Amazon Web Services",
            "category": "Company",
            "aliases": [
                "AWS"
            ],
            "description": "Cloud computing company involved in a strategic partnership for ethical AI research.",
            "location": null,
            "relationships": [
                {
                    "type": "employs",
                    "target": "Dr. Aisha Rahman"

## Summary
You have used the Gemini API to extract entities of predefined categories with their labels. You extracted every person, company, state, and country. You are not limited to these categories, as this should work with any category of your choice.

Please see the other notebooks in this directory to learn more about how you can use the Gemini API for other JSON related tasks.
